<a href="https://colab.research.google.com/github/OscarBastardo/saving-pots/blob/main/saving_pots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Saving Pots for Bastardo-Jones

## About this notebook

Rose and Oscar want to implement a *Saving Pots* system, whereby we split the money we save each month into pots, one for each area of our lives. So far we are considering
the following pots:
1. New house
2. Current house
3. New and current car
4. Children
5. Holidays
6. Wedding
7. Family
8. Investment

We may add more pots in the future.

We we will spend proportionally to the amount of money we save per month.

This notebook is intended to manage the saving pots.

## Saving accounts and investments
We decided it is best to have actual saving accounts where we can send some money *away*. So far we think two accounts, a higher-yielding one without withdrawals and a
lower-interest one that is easy to withdraw from.

Easy access:
- Current house
- New and current car
- Children
- Holidays
- Family

High-yield:
- New house
- Wedding

Additionally, money to invest should be sent somewhere else or converted into appreciating assets.

## Technical Challenges
1. How can we maintain the integrity of ground truth data?

## Back-of-the-envelope projection

In [1]:
import numpy as np
import pandas as pd
import pytest

# current savings as of 03/12/2023
rose_current_savings = 23600
oscar_current_savings = 10000
household_current_savings = rose_current_savings + oscar_current_savings

# money in a month
rose_take_home_month = 2500
oscar_take_home_month = 4500
household_income_month = rose_take_home_month + oscar_take_home_month

# money out a month
expenses_month = 3200

# share of expenses
rose_expenses_share = rose_take_home_month / household_income_month
oscar_expenses_share = oscar_take_home_month / household_income_month
assert pytest.approx(rose_expenses_share + oscar_expenses_share) == 1


# monthly surplus after paying expenses
rose_surplus_month = rose_take_home_month - expenses_month * rose_expenses_share
oscar_surplus_month = oscar_take_home_month - expenses_month * oscar_expenses_share
household_surplus_month = household_income_month - expenses_month
assert (
    pytest.approx(rose_surplus_month + oscar_surplus_month) == household_surplus_month
)

# pocket money a month
rose_pocket_money_month = 500
oscar_pocket_money_month = 500

# money saved a month
rose_savings_month = rose_surplus_month - rose_pocket_money_month
oscar_savings_month = oscar_surplus_month - oscar_pocket_money_month
household_savings_month = rose_savings_month + oscar_savings_month

# shares for current savings
current_savings_shares = {
    "car": 0.45,
    "wedding": 0.35,
    "new_house": 0.20,
}

assert pytest.approx(sum(current_savings_shares.values())) == 1

# shares of monthly savings
savings_shares = {
    "new_house": 0.30,
    "current_house": 0.15,
    "car": 0.15,
    "holiday": 0.10,
    "wedding": 0.10,
    "investment": 0.10,
    "children": 0.05,
    "family": 0.05,
}
assert pytest.approx(sum(savings_shares.values())) == 1

# saving pots
saving_pots = [
    "new_house",
    "current_house",
    "car",
    "holiday",
    "wedding",
    "investment",
    "children",
    "family",
]
# urgent saving pots
urgent_saving_pots = ["new_house", "car", "wedding"]

# savings projection
savings_df = pd.DataFrame(columns=saving_pots)
## current savings split
savings_df.loc[0] = [
    household_current_savings * current_savings_shares[pot]
    if pot in urgent_saving_pots
    else 0
    for pot in saving_pots
]
## monthly savings projection
time_range = range(1, 29)
for i in time_range:
    savings_df.loc[i] = [
        savings_df.loc[i - 1][pot] + household_savings_month * savings_shares[pot]
        for pot in saving_pots
    ]
# show savings projection
savings_df

,new_house,current_house,car,holiday,wedding,investment,children,family
0,6720.0,0.0,15120.0,0.0,11760.0,0.0,0.0,0.0
1,7560.0,420.0,15540.0,280.0,12040.0,280.0,140.0,140.0
2,8400.0,840.0,15960.0,560.0,12320.0,560.0,280.0,280.0
3,9240.0,1260.0,16380.0,840.0,12600.0,840.0,420.0,420.0
4,10080.0,1680.0,16800.0,1120.0,12880.0,1120.0,560.0,560.0
5,10920.0,2100.0,17220.0,1400.0,13160.0,1400.0,700.0,700.0
6,11760.0,2520.0,17640.0,1680.0,13440.0,1680.0,840.0,840.0
7,12600.0,2940.0,18060.0,1960.0,13720.0,1960.0,980.0,980.0
8,13440.0,3360.0,18480.0,2240.0,14000.0,2240.0,1120.0,1120.0
9,14280.0,3780.0,18900.0,2520.0,14280.0,2520.0,1260.0,1260.0
